In [72]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time, requests
import json
import numpy as np
import mysql.connector as mysql
from currency_converter import CurrencyConverter

In [73]:
import undetected_chromedriver as uc
driver = uc.Chrome()
driver.maximize_window()

In [74]:
c = CurrencyConverter()

In [75]:
mydb = mysql.connect(
  host="l6glqt8gsx37y4hs.cbetxkdyhwsb.us-east-1.rds.amazonaws.com",
  user="vhaithj2ezjri3a5",
  password="htf06l6xp0gsgoei",
  database = "bovesv3pjmkwzex5"
)

In [76]:
# driver=webdriver.Chrome(executable_path='D:\chromedriver.exe')
# driver.maximize_window()


In [77]:
# mail_address = 'windwalkerz42@gmail.com'
# password = '@Best'
# url = 'https://www.google.com/accounts/Login?hl=ja&continue=http://www.google.co.jp/'
# driver.get(url)

USD

In [78]:
def convertUsd(price):
    price = c.convert(price, 'USD', 'THB')
    return price

EUR

In [79]:
def convertEur(price):
    price = c.convert(price, 'EUR', 'THB')
    return price
    

  *** Function market price ***

GameSrig

In [80]:
def market_8(url, gameId, marketId):
    driver.get(url)
    try:
        findPrice = driver.find_element_by_xpath('//*[@id="blockB"]/div/div/div[2]/div').text
        price = float(findPrice.replace("฿",""))
    except:
        price = float(0)
    update = "UPDATE price_check SET now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

HRK

In [81]:
def market_9(url, gameId, marketId):
    driver.get(url)
    try:
        findPrice = driver.find_element_by_xpath('//*[@id="store_form"]/div/div[1]/div/div/div[2]')
        price = float(findPrice.text.strip("$"))
    except:
        price = float(0)
    price = round(convertUsd(price),2)
    update = "UPDATE price_check SET now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close


Kinguin

In [82]:
def market_10(url, gameId, marketId):
    driver.get(url)
    try:   
        findPrice = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[4]/div/div[2]/div/div/span[1]/span[2]').text
        price = float(findPrice.strip("€ "))
    except:
        price = float(0)
    price = round(convertEur(price),2)
    update = "UPDATE price_check SET now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

Steam

In [83]:
# def market_11(url, gameId, marketId, checkSteam):
    # driver.get(url)
    # if checkSteam == 0:
    #     driver.find_element_by_id("language_pulldown").click()
    #     driver.find_element_by_css_selector('#language_dropdown > div > a:nth-child(5)').click()
    #     time.sleep(3)
    #     dropdown = driver.find_element_by_id("ageYear")
    #     select = Select(dropdown)
    #     select.select_by_visible_text('2000')
    #     driver.find_element_by_css_selector('a.btnv6_blue_hoverfade:nth-child(1)').click()
    # time.sleep(2)
    # gameDescription = driver.find_element_by_css_selector('#game_highlights > div.rightcol > div > div.game_description_snippet').text
    # releaseDate = driver.find_element_by_xpath('//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]').text
    # developer = driver.find_element_by_xpath('//*[@id="genresAndManufacturer"]/div[1]/a').text
    # publisher = driver.find_element_by_xpath('//*[@id="genresAndManufacturer"]/div[2]/a').text
    # findTagsGame = driver.find_element_by_xpath('//*[@id="genresAndManufacturer"]/span')
    # tagsGame = findTagsGame.text.split(", ")
    # checkSpecMinimum = driver.find_elements_by_xpath('//*[@id="tabletGrid"]/div[1]/div[5]/div[2]/div[8]/div[1]/div/div/div[1]/ul/ul/li/strong')
    # spec_minimum_arr=[]
    # checkSpecMinimum =  int(2)
    # for i in checkSpecMinimum:
    #     if i.text == 'ระบบปฏิบัติการ:' or i.text =='หน่วยประมวลผล:' or i.text == 'หน่วยความจำ:' or i.text  == 'กราฟิกส์:' or i.text == 'พื้นที่จัดเก็บข้อมูล:':
    #         find_spec_minimum = driver.find_elements_by_xpath('//*[@id="tabletGrid"]/div[1]/div[5]/div[2]/div[8]/div[1]/div/div/div[1]/ul/ul/li['+str(checkSpecMinimum)+']/text()')
    #         checkSpecMinimum 
    # find_spec_recommend = driver.find_elements_by_xpath('//*[@id="tabletGrid"]/div[1]/div[5]/div[2]/div[8]/div[1]/div/div/div[2]/ul/ul/li')
    # spec_recommend_arr=[]
    # for i in find_spec_recommend:
    #     spec_recommend_arr.append(i.text)
    # findPrice = driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/div[1]/div[3]/div[1]/div[5]/div[2]/div[1]/div[2]/div/div[2]/div/div[1]').text
    # driver.find_element_by_class_name
    # price = float(findPrice.replace("฿","",).replace(",",""))
    # cursor = mydb.cursor()
    # update = "UPDATE price_check SET now_price = %s WHERE game_id  = %s AND market_id = %s"
    # values = [price, gameId, marketId]
    # cursor.execute(update,values)
    # mydb.commit()
    # cursor.close

    # cursor = mydb.cursor()
    # update = "UPDATE games_data SET release_date = %s, developer = %s, publisher = %s, game_description = %s WHERE game_id = %s"
    # values = [releaseDate, developer, publisher, gameDescription, gameId]
    # cursor.execute(update,values)
    # mydb.commit()
    # cursor.close

    # cursor = mydb.cursor()
    # cursor.execute("SELECT game_id FROM games_spec")
    # dataFromDB = cursor.fetchall()
    # count = int(0)
    # for i in dataFromDB:
    #     if i[0] == gameId:
    #         count += 1
    # if count == 0:
    #     insert ="INSERT INTO games_spec (game_id, bit_system_minimum, os_minimum, cpu_minimum,ram_minimum, vga_minimum, directx_minimum, storage_minimum, remark_minimum,bit_system_recommend, os_recommend, cpu_recommend, ram_recommend, vga_recommend, directx_recommend, storage_recommend, remark_recommend) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    #     values = []
    #     values.append(gameId)
    #     for i in spec_minimum_arr:
    #         values.append(i)
    #     for i in spec_recommend_arr:
    #         values.append(i)
    #     cursor.execute(insert,values)
    #     mydb.commit()
    # else:
    #     update = "UPDATE games_spec SET bit_system_minimum = %s, os_minimum = %s, cpu_minimum = %s, ram_minimum = %s, vga_minimum = %s, directx_minimum = %s, storage_minimum = %s, remark_minimum = %s, bit_system_recommend = %s, os_recommend = %s, cpu_recommend = %s, ram_recommend = %s, vga_recommend = %s, directx_recommend = %s, storage_recommend = %s, remark_recommend = %s WHERE game_id = %s"
    #     values = []
    #     for i in spec_minimum_arr:
    #         values.append(i)
    #     for i in spec_recommend_arr:
    #         values.append(i)
    #     values.append(gameId)
    #     cursor.execute(update,values)
    # mydb.commit()
    # cursor.close()
    
    # cursor = mydb.cursor()
    # cursor.execute("SELECT tag_name FROM tags_data")
    # dataFromDB  = cursor.fetchall()
    # for i in tagsGame:
    #     count = int(0)
    #     for x in myresult:
    #         if  x[0] == i:
    #             count += 1
    # if count == 0:
    #     insert ="INSERT INTO tags_data (tag_name) VALUES (%s)"
    #     values = [i]
    #     cursor.execute(insert,values)
    # mydb.commit()
    # cursor.close()

    # cursor = mydb.cursor()
    # cursor.execute("SELECT * FROM tags_data")
    # dataFromDB = cursor.fetchall()
    # listTagsId = []
    # for i in dataFromDB:
    #     for j in tagsGame:
    #         if i[1] == j:
    #             listTagsId.append(i[0])
    # cursor.close()

    # cursor = mydb.cursor()
    # cursor.execute("SELECT * FROM tags_data")
    # dataFromDB = cursor.fetchall()
    # for i in listTagsId:
    #     count = int(0)
    #     for j in dataFromDB:
    #         if j[0]
                

    #             if j ==  i[1]:
    #                 count += 1
    #         if count == 0:
    #             insert = "INSERT INTO games_tags_data (tag_id) VALUES (%s)"
    #             values = [j]



Steam price

In [84]:
def market_11(url, gameId, marketId):
    cursor = mydb.cursor()
    cursor.execute("SELECT steam_id FROM games_data WHERE game_id = "+str(gameId)+"")
    myresult = cursor.fetchall()
    response = requests.get(url)
    gameJson = response.json()
    price = 0
    try :
        if (gameJson[myresult[0][0]]['success'] == True) :
            gameDetail = gameJson[myresult[0][0]]['data']
            price = float(gameDetail['price_overview']['final_formatted'].replace("฿","").replace(",",""))
    except:
        print(myresult[0][0])
    finally:
        cursor = mydb.cursor()
        update = "UPDATE price_check SET now_price = %s WHERE game_id  = %s AND market_id = %s"
        values = [price,gameId,marketId]
        cursor.execute(update,values)
        mydb.commit() 
        cursor.close
    

Epic Games

In [85]:
def market_12(url, gameId, marketId, checkEpicgames):
    driver.get(url)
    try:
        if checkEpicgames == 0:
            driver.find_element_by_class_name('css-38hacf').click()
            checkEpicgames = True
    except:
        price = float(0)
    time.sleep(2)
    findPrice = driver.find_element_by_xpath('//*[@id="dieselReactWrapper"]/div/div[4]/main/div[2]/div/div/div/div[2]/div/div/aside/div/div/div[3]/div/div/span/div/span').text
    price =  float(findPrice.replace("THB","").replace(",",""))
    cursor = mydb.cursor()
    update = "UPDATE price_check SET now_price = %s WHERE game_id  = %s AND market_id = %s"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close
        


Gamivo

In [86]:
def market_13(url, gameId, marketId):
    driver.get(url)
    try:
        time.sleep(2)
        findPrice = driver.find_element_by_xpath('//*[@id="product"]/div[1]/div[3]/div/div[2]/app-product-top/div/div[3]/div[1]/div[1]/div[2]').text
        price = float(findPrice.strip("€ "))
        price = round(convertEur(price),2)
    except:
        price = float(0)
    cursor = mydb.cursor()
    update = "UPDATE price_check SET now_price = (%s) WHERE game_id  = (%s) AND market_id = (%s)"
    values = [price, gameId, marketId]
    cursor.execute(update,values)
    mydb.commit()
    cursor.close

In [87]:
cursor = mydb.cursor()
cursor.execute("SELECT game_id, market_id, market_url  FROM price_check")
myresult = cursor.fetchall()
checkEpicgames = 0
for i in myresult:
        print(i)
        if i[1] == 8:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_8(url, gameId, marketId)
        elif i[1] == 9:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_9(url, gameId, marketId)
        elif i[1] == 10:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_10(url, gameId, marketId)
        elif i[1] == 11:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_11(url, gameId, marketId)
        elif i[1] ==  12:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_12(url, gameId, marketId, checkEpicgames)
            checkEpicgames += 1
        elif i[1] ==  13:
            url = i[2]
            gameId = i[0]
            marketId = i[1]
            market_13(url, gameId, marketId)
mydb.commit()


(22, 8, 'https://www.gamesrig.com/god-of-war-steam-key')


C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_9760\176298565.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  findPrice = driver.find_element_by_xpath('//*[@id="blockB"]/div/div/div[2]/div').text


(22, 9, 'https://www.hrkgame.com/en/games/product/god-of-war')
(22, 10, 'https://www.kinguin.net/category/108174/god-of-war-steam-cd-key')
(22, 11, 'https://store.steampowered.com/api/appdetails?appids=1593500&l=thai')
(22, 12, 'https://www.epicgames.com/store/en-US/p/god-of-war')


C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_9760\3814315525.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('css-38hacf').click()
C:\Users\BoraBestz\AppData\Local\Temp\ipykernel_9760\3814315525.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  findPrice = driver.find_element_by_xpath('//*[@id="dieselReactWrapper"]/div/div[4]/main/div[2]/div/div/div/div[2]/div/div/aside/div/div/div[3]/div/div/span/div/span').text


(22, 13, 'https://www.gamivo.com/product/god-of-war-steam')
(49, 8, 'https://www.gamesrig.com/en/all-products/dying-light-2-steam-key')
(49, 9, 'https://www.hrkgame.com/en/games/product/dying-light-2-stay-human-global')
(49, 10, 'https://www.kinguin.net/category/109526/dying-light-2-steam-cd-key')
(49, 11, 'https://store.steampowered.com/api/appdetails?appids=534380&l=thai')
(49, 12, 'https://www.epicgames.com/store/en-US/p/dying-light-2-stay-human')
(49, 13, 'https://www.gamivo.com/product/dying-light-2-stay-human')
(50, 11, 'https://store.steampowered.com/api/appdetails?appids=559650&l=thai')
(50, 13, 'https://www.gamivo.com/product/witch-it')
(51, 11, 'https://store.steampowered.com/api/appdetails?appids=1172620&l=thai')
(51, 13, 'https://www.gamivo.com/product/sea-of-thieves-steam-gift')
(52, 8, 'https://www.gamesrig.com/en/monster-hunter-rise-steam-key')
(52, 9, 'https://www.hrkgame.com/en/games/product/monster-hunter-rise-deluxe-edition')
(52, 10, 'https://www.kinguin.net/categor

In [88]:
mydb.close()